In [1]:
import torch 
import numpy as np
import tqdm
import matplotlib.pyplot as plt 
import os
import json
import pandas as pd

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
from datasets import Dataset
import pandas as pd
import torch

c:\Users\mosta\anaconda3\envs\SQLtoTEXT\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
data_path = os.path.join(project_root, 'text_to_sql_generator', 'data', 'processed', 'text2sql_clean.csv')
cleaned_dataset = pd.read_csv(data_path)

In [4]:
cleaned_dataset.shape

(100000, 2)

In [5]:
cleaned_dataset.head()

,sql_prompt,sql
0,What is the total volume of timber sold by eac...,"SELECT salesperson_id, name, SUM(volume) as to..."
1,List all the unique equipment types and their ...,"SELECT equipment_type, SUM(maintenance_frequen..."
2,How many marine species are found in the South...,SELECT COUNT(*) FROM marine_species WHERE loca...
3,What is the total trade value and average pric...,"SELECT trader_id, stock, SUM(price * quantity)..."
4,Find the energy efficiency upgrades with the h...,"SELECT type, cost FROM (SELECT type, cost, ROW..."


In [6]:
df = pd.DataFrame(cleaned_dataset)

# Format input/output pairs
PREFIX = "translate to SQL: "
df["input_text"] = PREFIX + df["sql_prompt"]
df["target_text"] = df["sql"]


In [7]:
#  Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[["input_text", "target_text"]])

In [8]:
# Load model and tokenizer
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [9]:
# Tokenization function
def preprocess(example):
    inputs = tokenizer(example["input_text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(example["target_text"], max_length=256, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

In [10]:
tokenized_dataset = dataset.map(preprocess, batched=False)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-sql-finetuned",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=10,  # or less
    max_steps=5000,       # override if needed
    logging_dir="./logs",
    predict_with_generate=True,
)

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
trainer = Seq2SeqTrainer(
    model=model,              # Your model (T5, BART, etc.)
    args=training_args,       # The training arguments you defined
    train_dataset=tokenized_dataset,  # Your preprocessed training data
    tokenizer=tokenizer,      # Tokenizer for your model
    data_collator=data_collator,  # Handles batching and padding
)

C:\Users\mosta\AppData\Local\Temp\ipykernel_29908\3049639269.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
trainer.train()  # ✅ This is the correct method to call

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.161500
1000,0.495100
1500,0.438800
2000,0.394400
2500,0.375300
3000,0.372800
3500,0.334500
4000,0.328900
4500,0.340000
5000,0.348500


TrainOutput(global_step=5000, training_loss=0.4589886322021484, metrics={'train_runtime': 407.627, 'train_samples_per_second': 24.532, 'train_steps_per_second': 12.266, 'total_flos': 1353418014720000.0, 'train_loss': 0.4589886322021484, 'epoch': 0.1})

In [15]:
inference = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

output = inference(
    "translate to SQL: What is the total volume of timber sold by each salesperson?",
    max_length=100,
    num_beams=5,
    early_stopping=True
)
print(output[0]['generated_text'])

Device set to use cuda:0


SELECT salesperson, COUNT(*) as total_volume FROM sales GROUP BY salesperson;


In [16]:
model.save_pretrained("t5-sql-finetuned")
tokenizer.save_pretrained("t5-sql-finetuned")

('t5-sql-finetuned\\tokenizer_config.json',
 't5-sql-finetuned\\special_tokens_map.json',
 't5-sql-finetuned\\spiece.model',
 't5-sql-finetuned\\added_tokens.json',
 't5-sql-finetuned\\tokenizer.json')

In [17]:
print(os.listdir("./t5-sql-finetuned"))

['checkpoint-1000', 'checkpoint-1500', 'checkpoint-15500', 'checkpoint-16000', 'checkpoint-2000', 'checkpoint-2500', 'checkpoint-3000', 'checkpoint-3500', 'checkpoint-4000', 'checkpoint-4500', 'checkpoint-500', 'checkpoint-5000', 'config.json', 'generation_config.json', 'model.safetensors', 'special_tokens_map.json', 'spiece.model', 'tokenizer.json', 'tokenizer_config.json']


In [18]:
# Load the config file to see model details
with open("./t5-sql-finetuned/config.json", "r") as f:
    config = json.load(f)
    
# This might give clues about the original model
print(config.get("_name_or_path", ""))
print(config.get("model_type", ""))


t5


In [19]:
# Load your fine-tuned model
model = T5ForConditionalGeneration.from_pretrained("./t5-sql-finetuned").to("cuda")

# Use the standard T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("./t5-sql-finetuned")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from datasets import load_dataset

dataset = load_dataset("lamini/spider_text_to_sql")
train_data = dataset["train"]
val_data = dataset["validation"]
